In [2]:
! pip install bertopic

In [7]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
 
DATASET = 'clicr'
DATASET_PATH = '../datasets/'+DATASET+'/clinicalcases.tsv'

docs = []
with open(DATASET_PATH) as f:
    docs = f.readlines()



In [45]:
topic_model = BERTopic(top_n_words=30)
topics, probs = topic_model.fit_transform(docs)

topic_model.get_topics()[0]

[('eye', 0.020788212236196223),
 ('retinal', 0.01158268758095172),
 ('visual', 0.011464713156527811),
 ('corneal', 0.010752290766083233),
 ('ocular', 0.00904652307784903),
 ('vision', 0.008306469530144859),
 ('acuity', 0.008177445332635492),
 ('optic', 0.0074606626844610216),
 ('eyes', 0.007082028357956753),
 ('intraocular', 0.006357338503787615),
 ('lens', 0.006281248971165769),
 ('macular', 0.006084666379091111),
 ('in', 0.005624864044002651),
 ('the', 0.0055048389676119166),
 ('anterior', 0.0054194708502483075),
 ('topical', 0.005366657925933672),
 ('oct', 0.0050981835376480615),
 ('fundus', 0.005002267262298498),
 ('right', 0.004992430022931915),
 ('20', 0.004952654725584291),
 ('vitreous', 0.004942078271063204),
 ('of', 0.004934567106443933),
 ('iop', 0.004932660512533555),
 ('was', 0.00485530730587198),
 ('with', 0.004834874665314684),
 ('to', 0.004769546523738599),
 ('and', 0.004686271713909139),
 ('left', 0.004592636251608792),
 ('glaucoma', 0.00455730249622138),
 ('choroidal',

In [47]:
terms_by_topic = []
for topic in topic_model.get_topics().values():
    terms = []
    for item in topic:
        terms.append(item[0])
    terms_by_topic.append(terms)

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
MAX_DF = 0.95

# for i in range(len(sentences)):
#     topic_index = clustering_model.labels_[i]
#     documents_by_topic[topic_index].append(sentences[i])
#     topics[topic_index] += sentences[i] + " "

tfidf_model = TfidfVectorizer(max_df=MAX_DF)

document_term_matrix = tfidf_model.fit_transform(docs)
terms = tfidf_model.get_feature_names_out()
terms_by_topic = tfidf_model.inverse_transform(document_term_matrix)


In [63]:
terms_by_topic = tfidf_model.inverse_transform(document_term_matrix)

print(len(terms_by_topic)

97655

In [8]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim.corpora import Dictionary

data_samples2 = docs

tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')

for idx in range(len(data_samples2)):
    data_samples2[idx] = data_samples2[idx].lower()  # Convert to lowercase.
    data_samples2[idx] = tokenizer.tokenize(data_samples2[idx])  # Split into words.

# # Remove numbers, but not words that contain numbers.
data_samples2 = [[token for token in doc if not token.isnumeric()] for doc in data_samples2]
data_samples2 = [[token for token in doc if token not in stop_words] for doc in data_samples2]

# # Remove words that are only one character.
data_samples2 = [[token for token in doc if len(token) > 1] for doc in data_samples2]

dictionary = Dictionary(data_samples2)
corpus = [dictionary.doc2bow(text) for text in data_samples2]
texts = [[dictionary[word_id] for word_id, freq in doc] for doc in corpus]


In [ ]:
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary

TOP_WORDS = 30

dictionary = Dictionary(terms_by_topic)
corpus = [dictionary.doc2bow(t) for t in terms_by_topic]
texts = [[dictionary[word_id] for word_id, freq in c] for c in corpus]

cm = CoherenceModel(topics=terms_by_topic, texts=texts, corpus=corpus, dictionary=dictionary, coherence='c_v',topn=TOP_WORDS)

coherence = cm.get_coherence()
coherence_per_topic = cm.get_coherence_per_topic()

print('Coerencia: ', coherence)
print('Coerencia por Topico: ', coherence_per_topic)    

with open(OUTPUT_PATH + '/mntresults.txt', "a") as file:
    print('Limiar de distância: '+str(DISTANCE_THRESHOLD), file=file)   
    print('Qtdd de Tópicos: '+str(n_clusters), file=file)   
    print('Limiar TF-IDF: '+str(MAX_DF), file=file)  
    print('Coerência total: '+str(coherence), file=file)  
    print('Coerência por Tópicos: '+str(coherence_per_topic), file=file)  
    print('Top Words: '+str(TOP_WORDS), file=file)  

    print('----------------------------------------------------------------------------', file=file)  